In [ ]:
import flexynesis 
import torch, os
torch.set_num_threads(12)

# Modeling Drug Response Using Publicly Available Pharmocogenomics Datasets

We build models using CCLE data and make predictions on GDSC data. 

# Download Data

In [ ]:
if not os.path.exists("dataset1"):
    !wget -O ccle_gdsc.tgz "https://bimsbstatic.mdc-berlin.de/akalin/buyar/flexynesis-benchmark-datasets/dataset1.tgz" && tar -xzvf ccle_gdsc.tgz

## Define Data Importer 

In [ ]:
data_importer = flexynesis.DataImporter(path ='./dataset1/', 
                                       data_types = ['gex', 'cnv'], 
                                        concatenate=False, min_features=1000, top_percentile=0.2)

In [ ]:
train_dataset, test_dataset = data_importer.import_data()

In [ ]:
tuner = flexynesis.HyperparameterTuning(train_dataset, 
                                        model_class = flexynesis.DirectPred, 
                                        target_variables = "Erlotinib",
                                        batch_variables = None,
                                        config_name = "DirectPred", 
                                        n_iter=5)    

In [ ]:
model, best_params = tuner.perform_tuning()
torch.save(model, 'mymodel.pth')
#model = torch.load('mymodel.pth')
#model.eval()

In [ ]:
y_pred_dict = model.predict(test_dataset)

In [ ]:
metrics_df = flexynesis.evaluate_wrapper(y_pred_dict, test_dataset)
metrics_df

In [ ]:
for var in model.target_variables:
    model.compute_feature_importance(var, steps = 50)

In [ ]:
top_features = flexynesis.get_important_features(model, "Erlotinib", top=10)
top_features

In [ ]:
top_features = top_features.groupby('layer')['name'].apply(list).to_dict() #convert to dict

In [ ]:
df = flexynesis.subset_assays_by_features(test_dataset, top_features)
df

In [ ]:
response_category = flexynesis.split_by_median(test_dataset.ann['Erlotinib'])

In [ ]:
flexynesis.plot_scatter(df['cnv_EGFR'], test_dataset.ann['Erlotinib'])

In [ ]:
f = 'gex_EGFR'
flexynesis.plot_boxplot(response_category, df[f], 'Erlotinib', f)